# Phase de Nettoyage des données

Les différentes données récupérées ont tout d'abord besoin d'être nettoyées pour les besoins du projet. Si les données fournies par l'EUDA sont déjà très propres et ne nécéssiteront qu'un léger travail de tri, voire éventuellement quelques merge utiles sur certaines tables, ce n'est pas le cas des données scrapées à partir de pages Wikipedia qui vont nous demander un travail de nettoyage plus approfondi.

## Nettoyage des données scrapées depuis Wikipedia

On commence par générer notre dataframe à partir du scraping des données de [l'article Wikipedia sur la légalisation du Cannabis par Pays](https://en.wikipedia.org/wiki/Legality_of_cannabis). Le code détaillé et commenté se trouve dans le fichier `scraping.py`, on se contente ici d'appeler nos fonctions.


In [114]:
from Data.scraping import scraping as s

url = "https://en.wikipedia.org/wiki/Legality_of_cannabis"

table = s.scrape_second_wikipedia_table(url)
data, header = s.table_to_list(table)
legal_pays = s.make_pandas(data, header)

legal_pays.head()

,Recreational,Medical,Notes
Afghanistan,Illegal,Illegal,Main article: Cannabis in Afghanistan Producti...
Albania,Illegal,Legal,Main article: Cannabis in Albania Prohibited b...
Algeria,Illegal,Illegal,Main article: Cannabis in Algeria
Andorra,Illegal,Illegal,Main article: Cannabis in Andorra
Angola,Illegal,Illegal,Main article: Cannabis in Angola


On remarque alors que notre Dataframe contient plusieurs éléments qui nécéssitent d'être nettoyés, modifiés, voire supprimés. Dans l'ordre on peut citer:

1. **Supprimer la dernier colonne concernant les "Notes"**, qui peuvent nous être utile à des fins d'interprétation, ou encore pour aider à la classification ou à l'interprétation plus tard, mais qui en l'état seront inutiles à la statistique descriptive ou à la construction de modèles plus tard.

2. **Eliminer de notre dataframe les pays non-européens**, puisque les données de l'EUDA ne s'intéresse qu'au cas européeen, et que c'est là le coeur de notre sujet.

3. Enfin, il nous faudra **regrouper les différentes états des variables sous un format clair** En l'état, on dispose pour chaque pays d'une description parfois unique et idividuelle de la législation dans le pays donné. L'idéal serait de ne garder qu'une variable binaire: Légal/Illégal. Cependant la réalité est plus complexe dans certains pays, et il nous faudra alors faire des choix, et les justifier, pour essayer de ne garder qu'un nombre limité d'état pour chaque variable.

### Suppression de la colonne des Notes

Cela n'est pas bien compliqué, on utilise simplement les outils de la librairie Pandas:

In [115]:
import pandas as pd

legal_pays_clean = legal_pays.drop(columns = "Notes")

legal_pays_clean.head()

,Recreational,Medical
Afghanistan,Illegal,Illegal
Albania,Illegal,Legal
Algeria,Illegal,Illegal
Andorra,Illegal,Illegal
Angola,Illegal,Illegal


### Tri pour ne garder que les pays Européens

Cette opération n'est elle aussi pas très compliquée, on va se servir d'une table de correspondance scrapée sur le [site national de Statistiques Candiennes](https://www23.statcan.gc.ca/imdb/p3VD.pl?Function=getVD&TVD=1312110) (ce afin d'avoir le nom des pays en anglais), qui nous sera d'ailleurs aussi utile plus tard afin de faire correspondre chaque pays de l'Union européenne avec son code ISO 3316.

In [116]:
from Data.scraping import scraping as s

url = "https://www23.statcan.gc.ca/imdb/p3VD.pl?Function=getVD&TVD=1312110"

table = s.scrape_first_table(url)
data, header = s.table_to_list(table)
table_corres = s.make_pandas(data)

table_corres.head()

,0,1
Belgium,BE,BEL
Bulgaria,BG,BGR
Croatia,HR,HRV
Cyprus,CY,CYP
Czechia,CZ,CZE


On doit néanmoins ajouter à la main certains pays non-membres de l'Union Européenne, mais présent dans les données que nous avons à disposition (souvent des pays candidats comme la Turquie):

In [117]:
import pandas as pd

table_corres.loc["Austria"] = ["AT","AUT"]
table_corres.loc["Türkiye"] = ["TR","TUR"]
table_corres.loc["Norway"] = ["NO","NOR"]

table_corres.tail()

,0,1
Spain,ES,ESP
Sweden,SE,SWE
Austria,AT,AUT
Türkiye,TR,TUR
Norway,NO,NOR


On effectue ensuite le tri des pays européens selon la manière suivante, en supprimant de la table principale toutes les lignes qui n'ont pas d'équivalent dans la table de correspondance.

In [118]:
# On renomme certains pays pour correspondre aux données de l'EUDA 
legal_pays_clean.rename(index={"Turkey":"Türkiye"}, inplace=True)
legal_pays_clean.rename(index={"Czech Republic":"Czechia"}, inplace=True)

for index in legal_pays_clean.index:
    if index in table_corres.index:
        pass
    else:
        legal_pays_clean = legal_pays_clean.drop(index = index)

legal_pays_clean.head()


,Recreational,Medical
Austria,Possession for personal use decriminalized as ...,"Dronabinol, Sativex, Nabilone[25]"
Belgium,Decriminalized up to 3 g (1⁄10 oz) or cultivat...,Sativex[31]
Bulgaria,Illegal,Illegal
Croatia,Decriminalized[71],Legal[72]
Cyprus,Illegal,Legal[73]


### Recodage de chaque élément dans une variable ternaire

Il nous faut désormais faire un travail de recodage des informations contenues dans le tableau, afin de ne garder au final qu'une variable avec trois états possibles:

- `Legal` si la consommation, la possession et la vente sont légales et encadrées dans le pays
- `Illegal` si la consommation, la possession et la vente sont illégale et passibles de poursuites judiciaires
- `Decriminalized` si la consommation et la possession sont décriminalisées, mais la vente est illégale. On ne fait pas ici de différence entre la pénalisation ou non du cannabis (la seule différence étant que dans le cas ou ce dernier est pénalisé, la consommation et la possession sont toujours passibles d'une amende, mais pas de poursuites légales)

L'information est souvent contenue en clair dans notre tableau, mais perdue au milieu de précisions et de nuances propres à chaque pays. Par souci de simplicité, on appliquera les règles suivantes afin de recoder efficacement le tableau:

- **On ignorera toute limite de poids ou de nombres de plants**: Par exemple dans le cas de la République Tchèque, la possession de cannabis est décriminalisée jusqu'à 10g, et illégale au delà. On la considère donc simplement comme décriminalisée.

- **Dans le domaine médical, l'autorisation du Sativex vaut pour légalisation, celle du CBD non**: En effet, le Sativex contient du THC tout comme certains autres médicaments dérivés de cannabis, ce qui n'est pas le cas pour le CBD (ou alors en quantité très faible). En outre, au vu du fait que le CBD soit légal dans la plupart des pays, considérer son autorisation comme valant légalisation aurait pour conséquence d'appauvrir largement nos données.

- **On se tiendra à la stricte législation théorique, sans prendre en compte pour l'instant son application pratique**, cela aussi par souci de simplicité car la situation de chaque pays est unique et complexe. Par exemple, en Finlande, la possession et la consommation de cannabis est illégale *stricto sensu* et peut valoir une peine de prison. En réalité, les consommateurs s'en tirent souvent avec une simple amende; ce qui serait presque équivalent *en pratique* à une décriminalisation. Le fait est qu'il nous est impossible de trancher pour chaque pays à quel moment s'arrête la théorie et commence la pratique. Nous n'avons en effet ni la compétence sur le sujet, ni l'autorité nécessaire pour effectuer ce genre d'arbitrage, ce qui nous amène à ne prendre en compte que la législation en vigueur, et non son application réelle.
En revanche, il pourra nous être utile de tenir compte de ces différences d'application de la loi lors de l'analyse que nous ferons de nos données, mais pas dans la construction de celles-ci.

En prenant cela en compte, nous pouvons recoder le tableau de la manière suivante:

In [ ]:
# Recodage des information selon les règles fixées plus haut

for index in legal_pays_clean.index:

    content = legal_pays_clean.loc[index].values.flatten().tolist()
    
    for i in range(len(content)):
        if "Legal" in content[i] or "Sativex" in content[i]:
            content[i] = "Legal"
        elif "Illegal" in content[i] or "CBD" in content[i]:
            content[i] = "Illegal"
        elif "Decriminalized" in content[i] or "decriminalized" in content[i]:
            content[i] = "Decriminalized"

    legal_pays_clean.loc[index] = content

print(legal_pays_clean)

               Recreational                                            Medical
Austria      Decriminalized                                              Legal
Belgium      Decriminalized                                              Legal
Bulgaria            Illegal                                            Illegal
Croatia      Decriminalized                                              Legal
Cyprus              Illegal                                              Legal
Czechia      Decriminalized                                              Legal
Denmark             Illegal                                              Legal
Estonia      Decriminalized                          With a special permit[88]
Finland             Illegal                                              Legal
France              Illegal  A two-year trial program involving 3,000 patie...
Germany               Legal                                              Legal
Greece              Illegal                         

Puisqu'il reste après filtrage encore quelques valeurs qui n'ont pas de définition claire, nous n'avons d'autre choix que de les recoder à la main, en justifiant bien évidemment nos choix à chaque étape:

- En Estonie, l'usage du cannabis pour la médecine est autorisé avec "un permis spécial". Dans les faits, une recherche plus détaillée sur ce permis nous apprend qu'il n'est en réalité jamais délivré et qu'un seul patient a ce jour a été autorisé à recevoir du Sativex. Cependant, pour être cohérent avec notre règle fixée plus haut de ne prendre en compte que la législation théorique en vigueur, on choisit de considérer l'usage comme Légal, puisqu'il est théoriquement possible.

- En France, l'usage cannabis thérapeutique est aujourd'hui sujet d'expérimentation dans certains cas précis et très encadrés, mais reste en théorie illégal. On choisit donc de le coder comme étant illégal puisque les expérimentations encadrées n'ouvrent pas à une possibilité d'utilisation pour la population générale.

- En Roumanie, l'usage thérapeutique de dérivés de cannabis avec une teneur en THC inférieure à 0,2% est autorisée. Etant donnée que la plupart des pays (dont la France) autorisent le CBD à condition qu'il ne contienne pas plus de 0,3% de THC, on choisit de considérer l'usage thérapeutique en Roumanie comme illégal, puisqu'on s'est fixé comme règle plus haut que la légalisation du CBD ne valait pas celle du cannabis.

In [120]:
# Recodage à la main des informations plus ambigues

legal_pays_clean.loc['France','Medical'] = 'Illegal'
legal_pays_clean.loc['Estonia','Medical'] = 'Legal'
legal_pays_clean.loc['Romania','Medical'] = 'Illegal'

legal_pays_clean.head()

,Recreational,Medical
Austria,Decriminalized,Legal
Belgium,Decriminalized,Legal
Bulgaria,Illegal,Illegal
Croatia,Decriminalized,Legal
Cyprus,Illegal,Legal


On obtient donc finalement un tableau complètement nettoyé et exploitable pour nos analyses futures !

# Ajout de nouvelles tables
## Importation des tables
On ajoute ici de nouvelles tables provenant elles aussi de www.euda.europa.eu.
Celles-ci serviront à la fois à produire des **statistiques descriptives** mais également à nourrir notre modèle de **machine learning**.

On commence par l'importation des données **générales** sur les **soins et traitements** liés à la consommation de drogue.
On utilise le module **Pandas** pour passer de fichier .xslx (Excel) à des tables Pandas.
Par ailleurs, on ne garde à chaque fois que les colonnes qui nous intéressent en vue d'un **Merge**.

In [ ]:
import pandas as pd
import openpyxl


### On commence par les donnees generales
total = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Cannabis détaillé/jamais_traite_entrants/drogues_entrants.xlsx"
df1 = pd.read_excel(total, header=3)

# On ne garde que les colonnes qui nous intéressent
colonnes = ['Country', 'Year of Treatment', 'All substances', 'Cannabis']
df1 = df1[colonnes]


### On procede de la meme maniere pour les donnees concernant les personnes deja traitees auparavant
deja = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Cannabis détaillé/jamais_traite_entrants/deja_traite_entrants.xlsx"
df2 = pd.read_excel(deja, header=3)


# On ne garde que les colonnes qui nous intéressent
colonnes = ['Country', 'Year of Treatment', 'All substances', 'Cannabis']
df2 = df2[colonnes]




### On procede de la meme maniere pour les donnees concernant les personnes jamais traitees auparavant
jamais = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Cannabis détaillé/jamais_traite_entrants/jamais_traite_entrants.xlsx"
df3 = pd.read_excel(jamais, header=3)


# On ne garde que les colonnes qui nous intéressent
colonnes = ['Country', 'Year of Treatment', 'All substances', 'Cannabis']
df3 = df3[colonnes]


/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


## Premier Merge : table_base
Une fois les tables préparées, on les **Merge** pour créer une unique table : *table_base*.
Celle-ci nous servira de base pour effectuer un **Merge** de plus grandes ampleurs afin d'obtenir une grande table compilant toutes les données disponibles ici sur le sujet des **soins et traiements** liés à la consommation de drogues.

In [3]:
### On effectue le merge (left join) afin de grouper les données par pays


## On commence par un merge "intermédiaire"
# On commence par supprimer la colonne "year of treatment" pour df 2 car cette colonne apparaît dans les deux tables
df2 = df2.drop(columns=["Year of Treatment"])
intermediaire = pd.merge(df1, df2, how="left", on="Country")
# Il faut renommer certaines colonnes (noms en doublon)
intermediaire = intermediaire.rename(columns={
    "Country": "Country",
    "Year of Treatment": "Year of Treatment",
    "All substances_x": "All substances (global)",
    "Cannabis_x": "Cannabis (global)",
    "All substances_y": "All substances (deja traite.es)",
    "Cannabis_y": "Cannabis (deja traite.es)"
})


## On reproduit les mêmes étapes mais en "mergeant" intermediaire et df3 pour finaliser cette table de BaseException
# On commence par supprimer la colonne "year of treatment" pour df 2 car cette colonne apparaît dans les deux tables
df3 = df3.drop(columns=["Year of Treatment"])
table_base = pd.merge(intermediaire, df3, how="left", on="Country")
# Il faut renommer certaines colonnes (noms en doublon)
table_base = table_base.rename(columns={
    "Country": "Country",
    "Year of Treatment": "Year of Treatment",
    "All substances (global)": "All substances (global)",
    "Cannabis (global)": "Cannabis (global)",
    "All substances (deja traite.es)": "All substances (deja traite.es)",
    "Cannabis (deja traite.es)": "Cannabis (deja traite.es)",
    "All substances": "All substances (jamais traite.es)",
    "Cannabis": "Cannabis (jamais traite.es)",
})

# On enlève les lignes en trop à la fin (vides ou inutiles)
table_base = table_base.drop(table_base.index[29:32:])


## Vérifications préalables au Merge final
Avant de **Merge** toutes les tables disponibles sur le sujet, on vérifie que la colonne **"Country"** sur la base de laquelle se fera le Merge existe bien dans tous les *Dataframes*.

In [4]:
# Etape préalable : avant d'effectuer le merge, on vérifie que la colonne 'Country' existe bien pour toutes les tables


### On commence par importer le module OS qui va nous permettre de travailler de manière globale sur les fichier plutôt que de les modifier un à un
import os


### On commence par supprimer des tables potentiellement en double
# On créer un variable correspondant au dossier contenant nos tables
dossier = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Cannabis détaillé/jamais_traite_entrants"


# On sélectionne les fichiers excel qui sont en double
fichiers1 = [f for f in os.listdir(dossier) if f.endswith('.xlsx')]


# On sépare en deux listes les fichier ayant et n'ayant pas de colonne nommée "Country"
Liste_1 = []
Liste_2 = []
for fichier in fichiers1:
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier, header=3)
    if "Country" in df.columns:  
        Liste_1.append(chemin_fichier)
    else :
        Liste_2.append(chemin_fichier)
print(len(Liste_1), len(Liste_2))
# On a bien 39 tables dans la liste_1 et 0 dans Liste_2 : On est prêt.es pour le Merge !

/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default sty

36 0


/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default sty

## Etape préalable n°2 : supprimer les doublons (en matière de *Dataframe*)
On utilise le module **os** pour vérifier dans le titre des fichiers si apparaît le terme **"Copie"** qui permet donc, le cas échéant, de supprimer des fichiers en doublon.
L'existence de tels fichiers provient d'erreurs humaines ayant eu lieu lors du téléchargement des bases de données sur le site www.euda.europa.eu.

In [ ]:
# Etape préalable : avant d'effectuer le merge, on supprime les tables en double (triple, etc)


### On commence par importer le module OS qui va nous permettre de travailler de manière globale sur les fichier plutôt que de les modifier un à un
import os


### On commence par supprimer des tables potentiellement en double
# On crée un dossier tampon qui va conserver nos tables modifiées
# On sélectionne les fichiers excel qui sont en double
fichiers1 = [f for f in os.listdir(dossier) if "Copie" in f and f.endswith('.xlsx')]
for fichier in fichiers1:
    chemin_fichier = os.path.join(dossier, fichier)
    if os.path.isfile(chemin_fichier):  # Vérifie si c'est bien un fichier
        os.remove(chemin_fichier)  # Supprime le fichier

## Etape préalable n°3 : supprimer la colonne Year of Treatment
Pourquoi un tel choix ? Cette colonne apparaît dans chaque table et donne des informations redondantes. Elle est, sauf très rare exception, identique dans chacune des *Dataframes*. Ainsi, afin d'éviter d'alourdir la table finale et de faciliter le Merge (**problème avec les noms de colonnes redondants**), on fait le choix de supprimer de toutes les tables (**sauf pour table_base**) la colonne **"Year of Treatment"**.

In [6]:
### On va supprimer les colonnes "Year of Treatment" présentent dans toutes les tables pour éviter les doublons dans notre table finale
# A quelques exceptions prêt négligeables, ka date est toujours 2022, l'information est donc redondante
fichiers2 = [f for f in os.listdir(dossier) if f.endswith('.xlsx')]
# Chemin du nouveau dossier
chemin_dossier_nouveau = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Tampon"

# Créer le dossier s'il n'existe pas
os.makedirs(chemin_dossier_nouveau, exist_ok=True)

for fichier in fichiers2:
    chemin_fichier = os.path.join(dossier, fichier)
    chemin_fichier1 = os.path.join(chemin_dossier_nouveau, fichier)
    df = pd.read_excel(chemin_fichier, header=3)
    if "Year of Treatment" in df.columns :
        df = df.drop(columns=["Year of Treatment"])
    df.to_excel(chemin_fichier1, index=False) # On enregistre la modification, sinon le code ne sert à rien


/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default sty

Etape préalable n°4 : 

In [7]:
### On commence par ajouter le suffixe ".deja" aux colonnes des tables tables concernées
# On créer un variable correspondant au dossier contenant nos tables
dossier = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Tampon"


# On sélectionne les fichiers excel qui ont "deja" dans leur nom
fichiers1 = [f for f in os.listdir(dossier) if "deja" in f and f.endswith('.xlsx')]
fichiers = [f for f in os.listdir(dossier) if f.endswith('.xlsx')]


# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers1:
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    df.rename(columns={col: col + ".deja" if col not in ["Country", "Year of Treatment"] else col for col in df.columns}, inplace=True)
    df.to_excel(chemin_fichier, index=False) # On enregistre la modification, sinon le code ne sert à rien




### "jamais"
# On sélectionne les fichiers excel qui ont "jamais" dans leur nom
fichiers2 = [f for f in os.listdir(dossier) if "jamais" in f and f.endswith('.xlsx')]


# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers2:
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    df.rename(columns={col: col + ".jamais" if col not in ["Country", "Year of Treatment"] else col for col in df.columns}, inplace=True)
    df.to_excel(chemin_fichier, index=False) # On enregistre la modification, sinon le code ne sert à rien


### "global"
# On sélectionne les fichiers excel qui ont "global" dans leur nom
fichiers3 = [f for f in os.listdir(dossier) if "global" in f and f.endswith('.xlsx')]


# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers3:
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    df.rename(columns={col: col + ".global" if col not in ["Country", "Year of Treatment"] else col for col in df.columns}, inplace=True)
    df.to_excel(chemin_fichier, index=False) # On enregistre la modification, sinon le code ne sert à rien


test = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Tampon/age_traitement_global.xlsx"
test = pd.read_excel(test)




## Etape préalable 4bis :


**Statut pro**

*.pro*

In [ ]:
import pandas as pd
import os
dossier_2 = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Tampon"
# On sélectionne les fichiers excel qui ont "pro" dans leur nom
fichiers1 = [f for f in os.listdir(dossier_2) if "pro" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers1:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".pro" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien



,Country,Mean age.deja,&lt;15.deja,15 to 19.deja,20 to 24.deja,25 to 29.deja,30 to 34.deja,35 to 39.deja,40 to 44.deja,45 to 49.deja,50 to 54.deja,55 to 59.deja,60 to 64.deja,> 64.deja,Not known / missing.deja,Total.deja
0,Austria,28.0,2.0,66.0,128.0,82.0,83.0,38.0,26.0,16.0,9.0,6.0,2.0,0.0,0.0,458.0
1,Belgium,31.0,6.0,145.0,289.0,280.0,266.0,212.0,170.0,78.0,36.0,22.0,2.0,2.0,16.0,1524.0
2,Bulgaria,29.0,0.0,4.0,10.0,9.0,9.0,6.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,42.0
3,Croatia,31.0,0.0,8.0,42.0,49.0,26.0,36.0,21.0,4.0,4.0,0.0,0.0,0.0,NaN,190.0
4,Cyprus,30.0,0.0,3.0,17.0,14.0,18.0,9.0,2.0,2.0,0.0,0.0,1.0,NaN,NaN,66.0
5,Czechia,30.0,2.0,41.0,48.0,108.0,109.0,72.0,54.0,28.0,4.0,2.0,NaN,1.0,NaN,469.0
6,Denmark,NaN,0.0,146.0,373.0,368.0,233.0,145.0,82.0,66.0,24.0,20.0,9.0,7.0,0.0,1473.0
7,Estonia,36.0,NaN,NaN,2.0,1.0,3.0,5.0,2.0,1.0,1.0,NaN,NaN,NaN,NaN,15.0
8,Finland,32.0,0.0,1.0,6.0,5.0,6.0,1.0,3.0,2.0,2.0,0.0,0.0,0.0,0.0,26.0
9,France,29.0,18.0,439.0,923.0,793.0,579.0,390.0,230.0,152.0,80.0,21.0,16.0,5.0,0.0,3646.0


**sexe**

*.sex*

In [ ]:
# On sélectionne les fichiers excel qui ont "sex" dans leur nom
fichiers2 = [f for f in os.listdir(dossier_2) if "sex" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers2:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".sex" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien


**sdf**

*.sdf*

In [ ]:
# On sélectionne les fichiers excel qui ont "sex" dans leur nom
fichiers3 = [f for f in os.listdir(dossier_2) if "sdf" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers3:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".sdf" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien

**Niveau d'éducation**

*.lvl_educ*

In [ ]:
# On sélectionne les fichiers excel qui ont "niveau" dans leur nom
fichiers4 = [f for f in os.listdir(dossier_2) if "niveau" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers4:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".lvl_educ" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien

**Fréquence de consommation**

*.frqce_conso*

In [ ]:
# On sélectionne les fichiers excel qui ont "frequence" dans leur nom
fichiers5 = [f for f in os.listdir(dossier_2) if "frequence" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers5:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".frqce_conso" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien

**Nombre d'enfants**

*.enfants*

In [ ]:
# On sélectionne les fichiers excel qui ont "enfants" dans leur nom
fichiers6 = [f for f in os.listdir(dossier_2) if "enfants" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers6:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".enfants" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien

**Composition du ménage**

*.menage*

In [ ]:
# On sélectionne les fichiers excel qui ont "menage" dans leur nom
fichiers7 = [f for f in os.listdir(dossier_2) if "menage" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers7:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".menage" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien

**Age de l'entrant**

*.age_trait*

In [ ]:
# On sélectionne les fichiers excel qui ont "traitement" dans leur nom
fichiers8 = [f for f in os.listdir(dossier_2) if "traitement" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers8:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".age_trait" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien

**Age lors de la première consommation**

*.age_premiere_conso*

In [ ]:
# On sélectionne les fichiers excel qui ont "premier" dans leur nom
fichiers9 = [f for f in os.listdir(dossier_2) if "premier" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers9:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".age_premiere_conso" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien

**De qui a émané la demande de soins**
(institution judiciaire, proches, personne concernée, *etc*)

*.demande*

In [ ]:
# On sélectionne les fichiers excel qui ont "demande" dans leur nom
fichiers10 = [f for f in os.listdir(dossier_2) if "demande" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers10:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".demande" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien

In [48]:
### Maintenant que ces modifications sont faites, on effectue le merge
# On effectue autant de merge que nécessaire en créant de nouveaux suffixes à chaque fois
test=table_base
dossier = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Tampon"
fichiers = [f for f in os.listdir(dossier) if f.endswith('.xlsx')]
for fichier in fichiers[0:5] :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country", suffixes=('_a', '_b'))
for fichier in fichiers[6:12] :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country", suffixes=('_c', '_d'))
for fichier in fichiers[13:21] :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country", suffixes=('_e', '_f'))
for fichier in fichiers[22:26] :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country", suffixes=('_g', '_h'))
for fichier in fichiers[27:33] :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country", suffixes=('_i', '_j'))
for fichier in fichiers[34:36] :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country", suffixes=('_k', '_l'))
test

,Country,Year of Treatment,All substances (global),Cannabis (global),All substances (deja traite.es),Cannabis (deja traite.es),All substances (jamais traite.es),Cannabis (jamais traite.es),Inject.jamais,Smoke / inhale.jamais,...,Total.global,All substances.jamais,Opioids.jamais,Cocaine.jamais,Stimulants.jamais,Hypnotics and Sedatives.jamais,Hallucinogens.jamais,Volatile Inhalants.jamais,Cannabis.jamais,Other substances.jamais
0,Austria,2022.0,4709.0,1124.0,2756.0,458.0,1953.0,666.0,0.0,573.0,...,1124.0,1953.0,514.0,304.0,134.0,50.0,3.0,0.0,666.0,4.0
1,Belgium,2022.0,12144.0,3501.0,7376.0,1524.0,4151.0,1850.0,0.0,1490.0,...,3501.0,4151.0,216.0,1115.0,358.0,209.0,143.0,15.0,1850.0,29.0
2,Bulgaria,2022.0,1879.0,109.0,1164.0,42.0,605.0,66.0,0.0,65.0,...,109.0,605.0,84.0,37.0,105.0,18.0,2.0,0.0,66.0,25.0
3,Croatia *,2022.0,2315.0,478.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Cyprus,2022.0,884.0,340.0,308.0,66.0,535.0,251.0,0.0,251.0,...,340.0,535.0,83.0,125.0,74.0,1.0,NaN,1.0,251.0,NaN
5,Czechia,2020.0,7283.0,1072.0,3336.0,469.0,2519.0,539.0,NaN,497.0,...,1072.0,2519.0,470.0,28.0,1317.0,129.0,14.0,3.0,539.0,19.0
6,Denmark,2022.0,8241.0,3085.0,4408.0,1473.0,3368.0,1493.0,0.0,1445.0,...,3085.0,3368.0,206.0,648.0,136.0,49.0,7.0,5.0,1493.0,65.0
7,Estonia,2022.0,417.0,26.0,290.0,15.0,124.0,11.0,NaN,11.0,...,26.0,124.0,54.0,4.0,50.0,5.0,NaN,NaN,11.0,NaN
8,Finland,2021.0,427.0,62.0,263.0,26.0,164.0,36.0,0.0,35.0,...,62.0,164.0,66.0,2.0,42.0,18.0,0.0,0.0,36.0,0.0
9,France,2021.0,43078.0,19758.0,10628.0,3647.0,18195.0,10680.0,18.0,10194.0,...,21186.0,18195.0,2497.0,1911.0,494.0,472.0,183.0,15.0,10680.0,204.0


# Ajout de nouvelles tables : infractions à la loi

Ici, on va procéder de manière analogue à la partie précédente sur l'ajout des tables en matière de soins et de traitements liés à la consommation de drogues en Europe.

## Importation des tables
On utilise les modules **Pandas** et **OS** afin de passer les fichiers du format .xslx (Excel) au format Pandas.

## Vérification du type de fichier
On a rencontré au cours de notre travail des problèmes à causes de tables téléchargées au mauvais format, ce qui empêchait le code de tourner correctement.
Pour éviter de rencontre à nouveau ce type de souci, on vérifie que tous les fichiers sont bien des *.xlsx*.

In [81]:
import os
liste = []
### On crée une fonction pour récupérer le type d'un fichier basé sur son extension grâce au module os
def get_file_type(file_path):
    _, file_extension = os.path.splitext(file_path)
    return file_extension.lower()  # On met en minuscule pour éviter les problèmes



### On ajoute le type de chaque fichier à une liste
fichiers7 = [f for f in os.listdir(dossier) if f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers7:
    liste.append(get_file_type(fichier))

### On fait un test pour vérifier que tous les fichiers sont bien de .xlsx
all_equal = all(x == liste[0] for x in liste)
print(all_equal)

True


## On spécifie le type de drogue concernée

L'idée est ici de faire en sorte de rendre les tables plus **lisibles**, en particulier en vue d'un **merge** de celles-ci.
Le nom des colonnes étant redondant et peu lisible sans le nom de la table (2010, 2011, *etc*), l'idée est de donner via les **noms  de colonnes** des **clés de lecture**.

Pour cela, nous allons ajouter des informations au nom de colonne *via* le format suivant :
**année.détail1.détail2**



Voici la classification choisie :

**Détail1** :
Regroupe les différents types de drogues étudiées (glbal (*i.e.* toutes les drogues), cannabis (*i.e.* sous toutes ses formes), herbe, raisine, huile et plante).

**Détail2**
Regroupe les différents types d'infractions (vente, consommation, production, infractions/délinquants (total des différents types d'infractions)).

**Cas global**

In [64]:
import os
import pandas as pd

### On commence par ajouter le suffixe ".cannabis" aux colonnes des tables concernées
# On créer un variable correspondant au dossier contenant nos tables
dossier = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Delits_Crimes"
dossier_2 = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Tampon"
fichier = [f for f in os.listdir(dossier) if f.endswith('.xlsx')]

# On sélectionne les fichiers excel qui ont "Cannabis" dans leur nom
fichiers1 = [f for f in os.listdir(dossier) if "Cannabis" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers1:
    chemin_fichier = os.path.join(dossier, fichier)
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier)
# On utilise la ligne 3 (index 2) comme nouveaux titres de colonnes
    df.columns = df.iloc[2]  # Utiliser la troisième ligne comme titres 
# On supprime la ligne qui est devenue les titres de colonnes
    df = df.drop(index=2)
# On réinitialise les index après la suppression
    df.reset_index(drop=True, inplace=True)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".cannabis" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien


/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


**Herbe**

In [65]:
### "Herbe"
# On sélectionne les fichiers excel qui ont "Herbe" dans leur nom
fichiers2 = [f for f in os.listdir(dossier) if "Herbe" in f and f.endswith('.xlsx')]


# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers2:
    chemin_fichier = os.path.join(dossier, fichier)
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier)
# On utilise la ligne 3 (index 2) comme nouveaux titres de colonnes
    df.columns = df.iloc[2]  # Utiliser la troisième ligne comme titres 
# On supprime la ligne qui est devenue les titres de colonnes
    df = df.drop(index=2)
# On réinitialise les index après la suppression
    df.reset_index(drop=True, inplace=True)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".herbe" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien



/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


**Huile**

In [66]:
### "Huile"
# On sélectionne les fichiers excel qui ont "Huile" dans leur nom
fichiers3 = [f for f in os.listdir(dossier) if "Huile" in f and f.endswith('.xlsx')]


# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers3:
    chemin_fichier = os.path.join(dossier, fichier)
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier)
# On utilise la ligne 3 (index 2) comme nouveaux titres de colonnes
    df.columns = df.iloc[2]  # Utiliser la troisième ligne comme titres 
# On supprime la ligne qui est devenue les titres de colonnes
    df = df.drop(index=2)
# On réinitialise les index après la suppression
    df.reset_index(drop=True, inplace=True)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".huile" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien


/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


**Plante**

In [67]:
### "Plante"
# On sélectionne les fichiers excel qui ont "Plante" dans leur nom
fichiers5 = [f for f in os.listdir(dossier) if "Plante" in f and f.endswith('.xlsx')]


# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers5:
    chemin_fichier = os.path.join(dossier, fichier)
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier)
# On utilise la ligne 3 (index 2) comme nouveaux titres de colonnes
    df.columns = df.iloc[2]  # Utiliser la troisième ligne comme titres 
# On supprime la ligne qui est devenue les titres de colonnes
    df = df.drop(index=2)
# On réinitialise les index après la suppression
    df.reset_index(drop=True, inplace=True)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".plante" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien


/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


**Raisine**

In [68]:
### "Raisine"
# On sélectionne les fichiers excel qui ont "Raisine" dans leur nom
fichiers6 = [f for f in os.listdir(dossier) if "Raisine" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers6:
    chemin_fichier = os.path.join(dossier, fichier)
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier)
# On utilise la ligne 3 (index 2) comme nouveaux titres de colonnes
    df.columns = df.iloc[2]  # Utiliser la troisième ligne comme titres 
# On supprime la ligne qui est devenue les titres de colonnes
    df = df.drop(index=2)
# On réinitialise les index après la suppression
    df.reset_index(drop=True, inplace=True)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".raisine" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien


/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


**Global**

In [69]:
### "Global"
# On sélectionne les fichiers excel qui ont "global" dans leur nom
fichiers7 = [f for f in os.listdir(dossier) if "Global" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers7:
    chemin_fichier = os.path.join(dossier, fichier)
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier)
# On utilise la ligne 3 (index 2) comme nouveaux titres de colonnes
    df.columns = df.iloc[2]  # Utiliser la troisième ligne comme titres 
# On supprime la ligne qui est devenue les titres de colonnes
    df = df.drop(index=2)
# On réinitialise les index après la suppression
    df.reset_index(drop=True, inplace=True)
    df.columns = df.columns.astype(str)
    df = df.rename(columns={col: col + ".global" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien


/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/opt/conda/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default sty

## On spécifie le type d'infractions observées

**Consommation**

In [70]:
# On sélectionne les fichiers excel qui ont "Consommation" dans leur nom
fichiers1 = [f for f in os.listdir(dossier) if "Consommation" in f and f.endswith('.xlsx')]

# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers1:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    # On sélectionne les lignes de l'index 2 à 31 (inclus)
    df = df.iloc[2:32, :]
    df = df.rename(columns={col: col + ".conso" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien

**Délinquants**

In [71]:
### "Delinquants"
# On sélectionne les fichiers excel qui ont "Delinquants" dans leur nom
fichiers2 = [f for f in os.listdir(dossier) if "Deliquants" in f and f.endswith('.xlsx')]


# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers2:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    # On sélectionne les lignes de l'index 2 à 31 (inclus)
    df = df.iloc[2:32, :]
    df = df.rename(columns={col: col + ".delinquants" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien

**Infractions**

In [72]:
### "Infractions"
# On sélectionne les fichiers excel qui ont "Infractions" dans leur nom
fichiers3 = [f for f in os.listdir(dossier) if "Infractions" in f and f.endswith('.xlsx')]


# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers3:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    # On sélectionne les lignes de l'index 2 à 31 (inclus)
    df = df.iloc[2:32, :]
    df = df.rename(columns={col: col + ".infractions" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien

**Production**

In [73]:
### "Production"
# On sélectionne les fichiers excel qui ont "Production" dans leur nom
fichiers4 = [f for f in os.listdir(dossier) if "Production" in f and f.endswith('.xlsx')]


# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers4:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    # On sélectionne les lignes de l'index 2 à 31 (inclus)
    df = df.iloc[2:32, :]
    df = df.rename(columns={col: col + ".prod" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien

**Ventes**

In [74]:
### "Ventes"
# On sélectionne les fichiers excel qui ont "Ventes" dans leur nom
fichiers5 = [f for f in os.listdir(dossier) if "Ventes" in f and f.endswith('.xlsx')]


# On ajoute le suffixe aux colonnes souhaitées
for fichier in fichiers5:
    chemin_fichier_2 = os.path.join(dossier_2, fichier) 
    df = pd.read_excel(chemin_fichier_2)
    df.columns = df.columns.astype(str)
    # On sélectionne les lignes de l'index 2 à 31 (inclus)
    df = df.iloc[2:32, :]
    df = df.rename(columns={col: col + ".ventes" if col != "Country" else col for col in df.columns})
    df.to_excel(chemin_fichier_2, index=False) # On enregistre la modification, sinon le code ne sert à rien

## On passe au Merge final

On effectue un merge sur toutes les tables concernant les infractions à la loi ayant trait à la drogue (consommation, production, etc).
Le fait d'avoir modifier le nom des colonnes permet de donner un résultat plus lisible.

In [82]:
### Maintenant que ces modifications sont faites, on effectue le merge
# On effectue autant de merge que nécessaire en créant de nouveaux suffixes à chaque fois
test= pd.read_excel("/home/onyxia/work/Projet_Python_2A/Data/EUDA/Tampon/Nombre_Consommation_Cannabis.xlsx")
dossier = "/home/onyxia/work/Projet_Python_2A/Data/EUDA/Tampon"
for fichier in fichiers1 :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country")
for fichier in fichiers2 :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country")
for fichier in fichiers3 :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country")
for fichier in fichiers4 :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country")
for fichier in fichiers5 :
    chemin_fichier = os.path.join(dossier, fichier)
    df = pd.read_excel(chemin_fichier)
    test = pd.merge(test, df, how="left", on="Country")
table_infractions_legales = test
table_infractions_legales

,Country,2022.0.cannabis.conso_x,2021.0.cannabis.conso_x,2020.0.cannabis.conso_x,2019.0.cannabis.conso_x,2018.0.cannabis.conso_x,2017.0.cannabis.conso_x,2016.0.cannabis.conso_x,2015.0.cannabis.conso_x,2014.0.cannabis.conso_x,...,2016.0.cannabis.ventes,2015.0.cannabis.ventes,2014.0.cannabis.ventes,2013.0.cannabis.ventes,2012.0.cannabis.ventes,2011.0.cannabis.ventes,2010.0.cannabis.ventes,2009.0.cannabis.ventes,2008.0.cannabis.ventes,2007.0.cannabis.ventes
0,Austria,19909.0,19919.0,22778.0,24618.0,24680.0,28890.0,23977.0,20456.0,21272.0,...,7372.0,7086.0,6705.0,1595.0,1409.0,1428.0,1454.0,1460.0,1261.0,1833.0
1,Belgium,29572.0,28857.0,31540.0,31688.0,30148.0,29916.0,27080.0,28237.0,29825.0,...,6316.0,6460.0,6637.0,6074.0,5942.0,8341.0,8574.0,8892.0,9047.0,8200.0
2,Bulgaria,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3465.0,...,NaN,NaN,790.0,1424.0,800.0,625.0,627.0,641.0,438.0,314.0
3,Croatia,5159.0,5080.0,4810.0,5689.0,6788.0,6253.0,5493.0,4268.0,4866.0,...,1719.0,1620.0,1399.0,1799.0,1442.0,1790.0,1746.0,1519.0,931.0,1199.0
4,Cyprus,563.0,512.0,606.0,692.0,862.0,703.0,643.0,690.0,799.0,...,106.0,142.0,88.0,127.0,124.0,88.0,117.0,129.0,NaN,128.0
5,Czechia,6443.0,5253.0,4195.0,5293.0,6176.0,5019.0,4744.0,4224.0,500.0,...,1249.0,1029.0,1052.0,1073.0,870.0,733.0,632.0,661.0,608.0,463.0
6,Denmark,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Estonia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Finland,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,France,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5967.0,NaN,NaN


In [ ]:
# 